# 1.Import Library

In [1]:
from fastai import *
from fastai.text.all import *

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)

In [3]:
path.ls()

(#2) [Path('C:/Users/home/.fastai/data/imdb_sample/models'),Path('C:/Users/home/.fastai/data/imdb_sample/texts.csv')]

In [4]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


In [5]:
df['text'][1]

'This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is some merit in this view, but it\'s also true that no one forced Hindus and Muslims in the region to mistreat each other as they did around the time of partition. It seems more likely that the British simply saw the tensions between the religions and were clever enough to exploit them to their own ends.<br /><br />The result is that there is much cruelty and inhumanity in the situation and this is very u

In [6]:
from fastai.text import *
# Tokenization และ Numericalization
databunch_languagemodel = TextDataLoaders.from_csv(path=path, csv_fname='texts.csv')

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\data\transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


In [7]:
databunch_languagemodel.show_batch(max_n=3)

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\data\transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


,text,category
0,xxbos positive,"Loved the movie. I even liked most of the actors in it. But, for me Ms. Davis' very poor attempt at an accent, and her stiff acting really makes an otherwise compelling movie very hard to watch. Seriously if any other modern actor played the same role with the same style as Ms. Davis they would be laughed off the screen.<br /><br />I really think she 'phoned this one in'. Now if it had Myrna Loy or Ingrid Bergman playing the part of the wife I would have enjoyed it much more.<br /><br />I guess I just don't 'get' Bette Davis. I've always thought of her as an actor that 'plays herself' no matter what role she's in. The possible exception is Now Voyager.<br /><br />I'm sure many of the other reviewers will explain in careful (and I hope civil) detail how I am totally wrong on this. But, I'll continue to watch the movies she's in because I like the stories/writing/supporting casts, but, I'll always be thinking, of different actresses that could have done a better job."
1,xxbos positive,"This movie surprised me in a good way. From the box I got the impression that it was an action thriller but it was too funny to be a thriller, even though it was somewhat exciting.<br /><br />There's a lot of nice one-liners and funny situations in this movie and James Belushi was born to do Bill Manucci, he does a great job. The rest of the cast ain't half-bad either and especially Timothy Dalton is a treat.<br /><br />The story can get pretty confusing at times as new characters shows up during the film. Things get more complicated as nobody seldom tells the truth about things. If you don't pay attention things might get a bit messy in the end but I really liked it.<br /><br />Louis Morneau isn't all that well known but he has done a perfectly OK job with this one and I never really grew impatient while watching the movie.<br /><br />Made men is well worth checking out."
2,xxbos negative,"I am glad to read so many negative comments about the Tritter plot. Everyone I talk to says the same thing. They like House's gruff nature and his intelligence, but really dislike the vindictiveness of this continuing plot. It cuts into the real nature of the hospital story and makes everyone angry at police authority. It needs to have a more caring nature instead of the vindictiveness to everyone at the hospital. Also, there seems to be many questionable legal aspects to what Tritter is actually doing. He alone cannot freeze accounts and have the authority to stop doctors from writing prescriptions for patients. A lot of the vindictiveness he is showing also is hurting the very sick patients at the hospital and the is not a good storyline to portray. I voted the episode awful not because of the story itself, but when you insert the Trittor piece it turns me off and the rest of the plot gets hurt by it, People say they hate to watch the story lines anymore. Please change it. Get Tritter out."


In [ ]:
"""
สร้าง language_model_learner ด้วย AWD-LSTM เป็นโมเดลแบบ RNN, LSTM ไว้เราจะอธิบายต่อไป

AWD_LSTM เป็น Language Model ที่ได้ถูกเทรนมากับ Corpus ขนาดใหญ่ จากส่วนหนึ่งของ Wikipedia ชื่อ WikiText-103 มีขนาดใหญ่ถึง 103,227,021 Token และ 267,735 Vocab เรียบร้อยแล้ว

กำหนดค่า drop_mult (Dropout) เรื่อง Regularization ไว้เราจะอธิบายต่อไป
"""
databunch_languagemodel = TextDataLoaders.from_csv(path, 'texts.csv', text_col='text', is_lm=True, valid_pct=0.1)
learn = language_model_learner(databunch_languagemodel, AWD_LSTM, drop_mult=0.3, 
                               metrics=[accuracy, Perplexity()]).to_fp16()

# เทรนรอบแรก (Freeze)
learn.fit_one_cycle(1, 2e-2)

# Save
learn.save('1epoch')

# เทรนต่อ (Unfreeze)
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

# Save Encoder (สำคัญ! เพื่อเอาไปใช้ต่อในขั้นตอน Classifier)
learn.save_encoder('finetuned')

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.099240,3.961787,0.279921,52.551163,03:28


f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.680361,3.903892,0.286187,49.595089,03:53
1,3.622696,3.895651,0.287679,49.188046,03:52
2,3.461678,3.928939,0.284153,50.853001,03:54
3,3.170914,4.007859,0.280002,55.028915,03:43


In [ ]:
import warnings

# ใส่บรรทัดนี้ไว้ส่วนบนสุดของไฟล์ หรือก่อนจุดที่เกิด warning
warnings.filterwarnings("ignore")
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]
print("\n".join(preds))

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\

f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\fastai\callback\fp16.py:47: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  self.autocast,self.learn.scaler,self.scales = autocast('cuda', dtype=dtype),GradScaler('cuda', **self.kwargs),L()
f:\Python_NLP\learnning_nlp\.venv\Lib\site-packages\

i liked this movie because the characters are well - developed and very developed . The characters are very developed and have a great deal of time . The friendship is such a bloom and i will never forget anyone when i watch
i liked this movie because it 's like someone got a bunch of things to say about it , especially the idea that it is just not a type of movie like the John Lennon Story . Before you watch it


In [ ]:
dls_clas = TextDataLoaders.from_folder(path, valid='test', text_vocab=databunch_languagemodel.vocab)

# ดูตัวอย่างข้อมูล
dls_clas.show_batch(max_n=2)